# YOUR PROJECT TITLE

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **model analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `modelproject.py` file includes a function which could be used multiple times in this notebook.

Imports and set magics:

In [1]:
import numpy as np
from scipy import optimize
import sympy as sm
import matplotlib.pyplot as plt

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
from modelproject import OLGClass
model = OLGClass(do_print=False)

Introduction:

The Koopmans model serves as a basic framework for exploring fundamental economic issues. Within the Koopmans model, we make the assumption that an economy consists of a single consumer and producer who are the same individual, along with two goods. In certain instances, it is also known as a Robinson Crusoe economy.
The underlying presumption is that analyzing a one-agent economy will yield valuable insights into the operations of a real-world economy comprising numerous economic agents.


  

We begin by describing a scenario in which a Koopmans model could be applied:

Dwight, a former pilot and KGB agent from the Soviet Union, has transitioned to a livelihood as a freelance airplane photographer following the collapse of the Soviet Union. Over time, he has become a highly regarded photographer specializing in deserted island photography. Dwight was enjoying a state of great happiness, but it would soon take a turn for the worse... One day while flying over the uninhabited island known as Beets Island, Dwight encountered an engine malfunction and had to perform an emergency landing. Fortunately for Dwight, Beets Island lives up to its name. Upon finding 1200 beets, Dwight swiftly realizes that he can use them to successfully catch fish. If Dwight consumes all the beets, he won't be able to catch any fish. However, for each beet he uses, he will catch fish corresponding to the square root of the quantity of beets used for fishing. This is all the background we need to set up a Koopmans model.

Below we have described the various parameters and the model:

y  is the quantity of fish caught

z  is the quantity of beets

e1 is the initial amount of beets

e2 is the initial amount of fish

x1 is the preference for beets

x2 is the preference for fish

p1 is the price of beets

p2 is the price of fish

a  is the consumer's preference

i  is the income

pi is the profit

HUSK BESKRIV ALGORITME

# OLG (Overlapping Generations) model

Our model project examines the Overlapping Generations (OLG) model.

In [2]:
# a. create and simulate the model
par = model.par
sim = model.sim

p = 0.95 # survival probability
par.n = 0.02 # population growth
par.p = np.full(par.simT, p) 

model.simulate(do_print=False)


# b. plot the populations over time
plt.figure(figsize=(10, 6))
plt.plot(range(model.par.simT), model.par.pop_young, label="Young")
plt.plot(range(model.par.simT), model.par.pop_old, label="Old")
plt.xlabel("Period")
plt.ylabel("Population")
plt.legend()
plt.title("Population Dynamics Over Time")
plt.show()

NameError: name 'simulate_before_s' is not defined

# Walras equilibrium

## Analytical solution

If your model allows for an analytical solution, you should provide here.

You may use Sympy for this. Then you can characterize the solution as a function of a parameter of the model.

To characterize the solution, first derive a steady state equation as a function of a parameter using Sympy.solve and then turn it into a python function by Sympy.lambdify. See the lecture notes for details. 

## Numerical solution

You can always solve a model numerically. 

Define first the set of parameters you need. 

Then choose one of the optimization algorithms that we have gone through in the lectures based on what you think is most fitting for your model.

Are there any problems with convergence? Does the model converge for all starting values? Make a lot of testing to figure these things out. 

# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

# Conclusion

Add concise conclusion. 